In [ ]:
!pip install transformers datasets accelerate


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load CSV
data = pd.read_csv("data\Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv")

# Filter data jika diperlukan (contoh: hanya gunakan data tertentu berdasarkan flags)
# data = data[data['flags'] == 'use']

# Gabungkan instruction, intent, dan lainnya menjadi prompt
def create_prompt(row):
    instruction = row['instruction']
    intent = row['intent']
    category = row['category']
    # Gabungkan menjadi format prompt yang sesuai
    return f"Instruction: {instruction}\nIntent: {intent}\nCategory: {category}\n"

data['prompt'] = data.apply(create_prompt, axis=1)

# Split data menjadi train dan test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Simpan sebagai JSONL
train_data[['prompt', 'response']].to_json("data/train_data.jsonl", orient="records", lines=True)
test_data[['prompt', 'response']].to_json("data/test_data.jsonl", orient="records", lines=True)


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

from transformers import LlamaForCausalLM, LlamaTokenizer, TrainingArguments, Trainer
from datasets import load_dataset

# Load model and tokenizer
model_name = "model/llama-model"
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name)

# Load dataset
dataset = load_dataset("json", data_files={
    "train": "data/train_data.jsonl",
    "validation": "data/test_data.jsonl"
})

# Tokenize data
def preprocess_function(examples):
    inputs = examples["prompt"]
    outputs = examples["response"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(outputs, max_length=512, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
    fp16=True  # Enable if using GPU
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

# Fine-tune model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("model/fine_tuned_llama")
tokenizer.save_pretrained("model/fine_tuned_llama")


NameError: name '_C' is not defined

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

# Generate a response
def chat(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
print(chat("Apa itu AI?"))
